In [1]:
### Import all the necessary modules
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
### Import Data
data = pd.read_csv("MergeDataPercentile.csv")
data = data[data['Pos'] != '0']
data.dropna(axis=0,inplace=True,subset=['Advanced StatsDRB%'])

hold_out = data[data['Highest Level Reached_x'] == '2017-18']
keep_data = data[data['Highest Level Reached_x'].isin(['NBA','International'])]


In [3]:
## Which Features to keep
hold_out.columns.values
feature_set_1 = ['Per GameGP', 'Per GameGS', 'Per GameMIN','Misc StatsOWS', 'Misc StatsDWS','Advanced StatsTS%', 'Advanced StatseFG%',
       'Advanced StatsORB%', 'Advanced StatsDRB%', 'Advanced StatsTRB%',
       'Advanced StatsAST%', 'Advanced StatsTOV%', 'Advanced StatsSTL%',
       'Advanced StatsBLK%', 'Advanced StatsUSG%','Advanced StatsPPR', 'Advanced StatsPPS',
       'Advanced StatsORtg', 'Advanced StatsDRtg', 'Advanced StatsPER','Pos']
feature_set_2 = ['Per GameMIN','Misc StatsOWS','Misc StatsDWS','Pos','Per GameFG%', 'Per Game3PM',
        'Per Game3P%', 'Per GameFTM',
       'Per GameFT%', 'Per GameDEF', 'Per GameTRB',
       'Per GameAST', 'Per GameSTL', 'Per GameBLK',
       'Per GameTOV', 'Per GamePTS']
feature_set_3 = list(set(feature_set_1).union(set(feature_set_2)))
y_col = ["Highest Level Reached_x"]
X = keep_data[feature_set_1]
y = keep_data[y_col]
le = LabelEncoder()
y = le.fit_transform(y)

C:\Users\Abhijit\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
## Encode the position, encode the response variable
X_dummies = pd.get_dummies(X,columns=['Pos'])
#y_dummies = pd.get_dummies(y)


In [5]:
### Split into train, test, validation sets
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.3, random_state=42)
X_test,X_val,y_test,y_val = train_test_split(X_test, y_test, test_size=0.1, random_state=42)

In [6]:
### penalty parameters to search over


In [7]:
### how to find the best logistic classifier
best_F1 = 0
best_classifier = None
for k in np.arange(1,201,5):
    classifier = KNeighborsClassifier(n_neighbors=k,weights = 'distance')
    classifier.fit(X_train,y_train)
    val_pred = classifier.predict(X_val)
    score = np.sum(val_pred==y_val)/val_pred.shape[0]
    print(score)
    if score >= best_F1:
        best_classifier = classifier
        best_F1 = score
print('Found Best Classifier!')

        

0.831715210356
0.859223300971
0.878640776699
0.877022653722
0.881877022654
0.877022653722
0.880258899676
0.878640776699
0.880258899676
0.880258899676
0.878640776699
0.877022653722
0.880258899676
0.877022653722
0.877022653722
0.878640776699
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.875404530744
0.877022653722
0.875404530744
0.875404530744
0.877022653722
0.877022653722
0.877022653722
0.877022653722
0.877022653722
0.877022653722
0.875404530744
Found Best Classifier!


In [8]:
### Check for overfit
pred_test = best_classifier.predict(X_test)
test_f1 = f1_score(y_test,pred_test,average='micro')
test_f1

0.87517985611510796

In [9]:
## We seem good here, now lets predict on the class of 2017-2018
X_hold_out = hold_out[feature_set_1]
X_hold_out = pd.get_dummies(X_hold_out,columns=['Pos'])
#y_hold_out = hold_out[y_col]
y_hold_out = best_classifier.predict(X_hold_out)

In [10]:
best_classifier.predict_proba(X_hold_out)

array([[ 0.94663269,  0.05336731],
       [ 0.84169817,  0.15830183],
       [ 1.        ,  0.        ],
       ..., 
       [ 0.8060186 ,  0.1939814 ],
       [ 0.90889718,  0.09110282],
       [ 0.95081948,  0.04918052]])

In [11]:
predictions = pd.DataFrame(data = y_hold_out,columns = ['prediction'])
predictions['Name'] = hold_out['Name'].values
predictions[['International','NBA']] = pd.DataFrame(best_classifier.predict_proba(X_hold_out))

In [12]:
predictions[predictions['prediction']=='NBA']

C:\Users\Abhijit\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [ ]:
best_classifier